# 1. オペレーティングシステム 演習 15
#  シグナル

名前と学生証番号を書け. Enter your name and student ID.

 * 名前 Name:
 * 学生証番号 Student ID:

# 2. sigaction使用例
* SIGINT を5回受け取ると終了するプログラム
* SIGINT を送るには端末から Ctrl-C を入力するので, Jupyter notebook上では実行せず, 端末を用いて行うこと

In [ ]:
%%writefile sigint.c
#com 1
//% file: sigint.c
//% cmd: gcc -Wall -Wextra -o sigint sigint.c

#include <err.h>
#include <signal.h>
#include <stdio.h>
#include <sys/types.h>
#include <unistd.h>

volatile long n_sigints = 0;

void sigint_action(int sig, siginfo_t * info, void * arg) {
  (void)sig; (void)info; (void)arg;
  printf("got sigaction %ld\n", n_sigints);
  fflush(stdout);
  n_sigints++;
}

int main() {
  struct sigaction act;
  act.sa_sigaction = sigint_action;
  sigemptyset(&act.sa_mask);
  act.sa_flags = SA_SIGINFO;
  if (sigaction(SIGINT, &act, 0) == -1) err(1, "sigaction");
  pid_t pid = getpid();
  printf("please send me (pid = %d) SIGINT signal 5 times\n", pid);
  fflush(stdout);
  while (n_sigints < 5) { }
  return 0;
}

In [ ]:
gcc -Wall -o sigint sigint.c

* 実行は端末で
```
./sigint
```
として Ctrl-C を5回入力せよ

# 3. Segmentation Fault (SIGSEGV)とその処理
* Segmentation Faultは不正な領域へのアクセス時に発生する
* これも実はシグナルの一種であって, sigaction でその処理を設定できる(設定しなければデフォルトの動作として, プログラムが終了する)
* 以下は, SIGSEGVに対する動作を設定した後で, mprotectで領域への書き込みを禁止, 書き込みを行った時にそこを書き込み許可するプログラム


In [ ]:
%%writefile sigsegv.c
#com 1
//% file: sigsegv.c
//% cmd: gcc -Wall -Wextra -o sigsegv sigsegv.c

#include <assert.h>
#include <err.h>
#include <signal.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/mman.h>
#include <sys/types.h>
#include <unistd.h>

volatile long n_sigints = 0;

void sigsegv_action(int sig, siginfo_t * sinfo, void * arg) {
  (void)arg;
  void * addr = sinfo->si_addr;
  size_t page_sz = 4096;
  assert(sig == SIGSEGV);
  printf("got segv at %p\n", addr);
  fflush(stdout);
  void * page_addr = (void*)((long)addr & ~(page_sz - 1));
  if (mprotect(page_addr, page_sz, PROT_READ|PROT_WRITE) == -1)
    err(1, "mprotect");
}

int main() {
  struct sigaction act;
  act.sa_sigaction = sigsegv_action;
  sigemptyset(&act.sa_mask);
  act.sa_flags = SA_SIGINFO;
  if (sigaction(SIGSEGV, &act, 0) == -1) err(1, "sigaction");
  size_t page_sz = 4096;
  size_t sz = 4096;
  char * a = aligned_alloc(page_sz, sz);
  if (!a) err(1, "aligned_alloc");
  printf("a = %p\n", a);
  printf("a[0] = 100\n");
  a[0] = 100;
  if (mprotect(a, sz, PROT_READ) == -1) err(1, "mprotect");
  printf("a[0] = 200\n");
  a[0] = 200;
  printf("OK\n");
  return 0;
}

In [ ]:
gcc -Wall -o sigsegv sigsegv.c

In [ ]:
./sigsegv